# Pfam Protein Sequence Classification with Tensorflow and Keras

Adapted from Saleh Alkhalifa. [Machine Learning in Biotechnology and Life Sciences](https://github.com/PacktPublishing/Machine-Learning-in-Biotechnology-and-Life-Sciences).

## Overview

This tutorial provides a step-by-step guide for building a deep learning model to classify protein sequences by their family accessions using the Pfam dataset. Leveraging TensorFlow and Keras, we will:

- Process amino acid sequences into a format suitable for neural networks.
- Train a deep learning model to predict protein family classifications.
- Evaluate performance and interpret results to understand sequence-function relationships.

By the end, you will have a working model capable of automating protein family annotation, with applications in bioinformatics, drug discovery, and evolutionary biology.

### Pfam

The [Pfam](http://pfam.xfam.org/) dataset consists of several columns, as follows:

- *Family_id*: The name of the family that the seqeunce belongs to (for example, filamin).
- *Family Accession*: The class or output that our model will aim to predict.
- *Sequence*: The amino acid sequence we will use as input for our model

Pfam: The protein families database in 2021: J. Mistry, S. Chuguransky, L. Williams, M. Qureshi, G.A. Salazar, E.L.L. Sonnhammer, S.C.E. Tosatto, L. Paladin, S. Raj, L.J. Richardson, R.D. Finn, A. Bateman
Nucleic Acids Research (2020) doi: 10.1093/nar/gkaa913

### TensorFlow and Keras

- **[TensorFlow](https://www.tensorflow.org/)** is an end-to-end open source platform for machine learning. It has a comprehensive, flexible ecosystem of tools, libraries and community. 
- **[Keras](https://keras.io/)** is a deep learning API written in Python, running on top of the machine learning platform TensorFlow.


## Learning Objectives

- Learn how to preprocess protein sequence data for deep learning
    - Encode amino acid sequences
    - Pad sequences to uniform length
    - Convert labels to categorical format
- Build and train a deep learning model for sequence classification using TensorFlow/Keras
    - Implement embedding layers
    - Use bidirectional LSTM architecture
    - Apply dropout for regularization
- Evaluate model performance using standard metrics
    - Analyze accuracy and loss curves
    - Interpret classification reports
    - Visualize confusion matrices

### Tasks to be completed

- Download and prepare Pfam dataset
- Preprocess protein sequences and labels
- Build and train deep learning model
- Evaluate model performance
- Generate predictions and visualize results

## Prerequisites

- A working Python environment and familiarity with Python
- Basic understanding of machine learning concepts
- Familiarity with pandas and numpy libraries
- Knowledge of basic statistical concepts

## Get Started

- Please select kernel "conda_tensorflow2_p310" from SageMaker notebook instance.
- Import the necessary libraries and download the needed data.

### Import necessary libraries

In [ ]:
# Import the pyplot module from matplotlib for plotting.
import matplotlib.pyplot as plt

# Import the numpy library for numerical operations, often used for array manipulations.
import numpy as np

# Import the pandas library for data manipulation and analysis, especially for DataFrames.
import pandas as pd

# Import the seaborn library for statistical data visualization, built on top of matplotlib.
import seaborn as sns

# Import specific layers from keras.layers for building neural networks.
from keras.layers import (
    LSTM, # Import LSTM layer for Long Short-Term Memory networks.
    Bidirectional, # Import Bidirectional layer for bidirectional processing in RNNs.
    Conv1D, # Import Conv1D layer for 1D convolutional neural networks.
    Dense, # Import Dense layer for fully connected neural networks.
    Dropout, # Import Dropout layer for regularization to prevent overfitting.
    Embedding, # Import Embedding layer for creating word embeddings.
    Flatten, # Import Flatten layer to flatten the input tensor.
    Input, # Import Input layer to instantiate a Keras tensor.
    MaxPooling1D, # Import MaxPooling1D layer for 1D max pooling.
)

# Import the Model class and Sequential class from keras.models to define neural network models.
from keras.models import Model, Sequential

# Import the pad_sequences function from keras.preprocessing.sequence for padding sequences to the same length.
from keras.preprocessing.sequence import pad_sequences

# Import the l2 regularizer from keras.regularizers for applying L2 regularization to layers.
from keras.regularizers import l2

# Import classification_report and confusion_matrix from sklearn.metrics for model evaluation.
from sklearn.metrics import classification_report, confusion_matrix

# Import train_test_split from sklearn.model_selection for splitting data into training and testing sets.
from sklearn.model_selection import train_test_split

# Import LabelEncoder from sklearn.preprocessing for encoding categorical labels into numerical form.
from sklearn.preprocessing import LabelEncoder

# Import the tensorflow library, the main deep learning framework.
import tensorflow as tf

# Import EarlyStopping callback from tensorflow.keras.callbacks to stop training early when validation loss stops improving.
from tensorflow.keras.callbacks import EarlyStopping

# Import to_categorical from tensorflow.keras.utils for one-hot encoding of categorical variables.
from tensorflow.keras.utils import to_categorical

# Import Sequential model from tensorflow.keras.models for linear stack of layers.
from tensorflow.keras.models import Sequential

# Import Embedding, Bidirectional, LSTM, Dropout, and Dense layers from tensorflow.keras.layers for building neural networks.
from tensorflow.keras.layers import Embedding, Bidirectional, LSTM, Dropout, Dense

# Set the default style for seaborn plots to "darkgrid" for better visualization.
sns.set_style("darkgrid")

# Import the os module to interact with the operating system, used here for environment variables.
import os
# Force CPU usage if no GPU detected
os.environ['CUDA_VISIBLE_DEVICES'] = '-1'  # Set environment variable to force TensorFlow to use CPU only by disabling GPU visibility.

# Import the tensorflow library again (it's already imported above, this line might be redundant).
import tensorflow as tf

# Print the number of GPUs available to TensorFlow, useful for checking GPU setup.
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

import warnings
# Filter out and ignore all warning messages that might be generated during the execution of the code.
warnings.filterwarnings('ignore')

# Note: The first time run these imports may take some time, because Matplotlib is building the font cache.

### Download dataset

We load the dataset directly into a pandas DataFrame for analysis, streamlining the data ingestion process.

In [ ]:
# Define the base URL for the dataset files hosted on GitHub.
URL = "https://raw.githubusercontent.com/PacktPublishing/Machine-Learning-in-Biotechnology-and-Life-Sciences/main/datasets/dataset_pfam"

# Initialize an empty list called 'files' to store DataFrames.
files = []

# Loop 8 times to read in 8 different CSV files.
for i in range(8):
    # Read each CSV file from the specified URL pattern into a pandas DataFrame.
    # The filename is constructed by appending 'dataset_pfam_seq_sd' and the loop index (i+1), followed by '.csv'.
    # 'index_col=None' prevents pandas from using the first column as index.
    # 'header=0' sets the first row as the header of the DataFrame.
    df = pd.read_csv(f"{URL}/dataset_pfam_seq_sd{i+1}.csv", index_col=None, header=0)
    
    # Append the DataFrame read from the CSV file to the 'files' list.
    files.append(df)

# Concatenate all DataFrames stored in the 'files' list into a single DataFrame 'df'.
# 'axis=0' concatenates along rows (vertically).
# 'ignore_index=True' resets the index of the resulting DataFrame to a new sequential index.
df = pd.concat(files, axis=0, ignore_index=True)

# Print the shape (number of rows and columns) of the concatenated DataFrame 'df'.
df.shape

### Examine the data

To quickly inspect your dataset's structure and first few rows, use df.head(). This displays the initial 5 entries by default, giving you a snapshot of column names, data types, and sample values—helpful for early data exploration.

In [ ]:
df.head()

To identify missing values in your dataset, use df.isna().sum(). This command checks for NaN values across each column in the DataFrame df and returns the total count of missing entries per column, helping you quickly assess data completeness.

In [ ]:
df.isna().sum()

To get the top 10 most frequent family IDs from a DataFrame column by grouping the data by family_id, counts occurrences of each ID, and returns the 10 most abundant families in descending order.

In [ ]:
# 1. Select the 'family_id' column from DataFrame df
# 2. Group the data by the same 'family_id' values
# 3. For each group, count the occurrences of each family_id (value_counts)
# 4. From these counts, select the top 10 most frequent family_ids (nlargest)
df["family_id"].groupby(df["family_id"]).value_counts().nlargest(10)

To get the top 10 most frequent family accessions, group the DataFrame column df['family_accession'] by itself, count occurrences of each value, and extract the 10 largest

In [ ]:
# Breakdown of operations:
# 1. Select the 'family_accession' column from DataFrame df
# 2. Group the data by the same 'family_accession' values (creates groups of identical accessions)
# 3. Count occurrences of each accession within its own group (redundant in this case)
# 4. Select the top 10 most frequent accessions
df["family_accession"].groupby(df["family_accession"]).value_counts().nlargest(10)

Generate a histogram of sequence length frequencies to analyze their distribution in the dataset.

In [ ]:
# Create a distribution plot of protein sequence lengths using seaborn's displot
sns.displot(
    df["sequence"].apply(lambda x: len(x)),  # Calculate length of each sequence in 'sequence' column
    bins=75,            # Use 75 bins to show distribution granularity
    height=4,           # Set plot height to 4 inches  
    aspect=2            # Set aspect ratio (width = height * aspect = 8 inches)
)

The mean sequence length can be obtained by first determining the character count of each entry in the 'sequence' column using str.len(), then calculating the average of these lengths with .mean().

In [ ]:
# Calculate the average length of protein sequences in the DataFrame
# 1. df["sequence"] - Selects the 'sequence' column containing protein sequences
# 2. .str.len() - Computes the length of each string in the sequence column
# 3. .mean() - Calculates the arithmetic mean of all sequence lengths

df["sequence"].str.len().mean()

The minimum sequence length can be obtained by first determining the character count of each entry in the 'sequence' column using str.len(), then calculating the average of these lengths with .min().

In [ ]:
# Calculate the minimum length of protein sequences in the DataFrame
# 1. df["sequence"] - Accesses the 'sequence' column containing protein sequences
# 2. .str.len() - Computes the length of each protein sequence (number of amino acids)
# 3. .min() - Returns the smallest sequence length found in the column

df["sequence"].str.len().min()

The maximum sequence length can be obtained by first determining the character count of each entry in the 'sequence' column using str.len(), then calculating the average of these lengths with .max().

In [ ]:
# Calculate the maximum length of protein sequences in the DataFrame
# 1. df["sequence"] - Selects the 'sequence' column containing protein sequences (amino acid strings)
# 2. .str.len() - Computes the length (number of amino acids) for each sequence
# 3. .max() - Identifies and returns the largest sequence length in the dataset

df["sequence"].str.len().max()

The mediam sequence length can be obtained by first determining the character count of each entry in the 'sequence' column using str.len(), then calculating the average of these lengths with .median().

In [ ]:
# Calculate the median length of protein sequences in the DataFrame
# 1. df["sequence"] - Selects the column containing protein sequence strings
# 2. .str.len() - Computes the length (number of amino acids) for each sequence
# 3. .median() - Calculates the median value of all sequence lengths

df["sequence"].str.len().median()

Retrieve all family accessions where the count exceeds 1200, filtering for high-frequency records.

In [ ]:
# Filter the DataFrame to only include protein families with more than 1200 sequences
# 1. df.groupby("family_accession") - Groups the DataFrame by protein family accession IDs
# 2. .filter(lambda x: len(x) > 1200) - Keeps only groups with more than 1200 sequences
#    - lambda x: len(x) counts the number of sequences in each family group
#    - > 1200 specifies the minimum count threshold
# 3. Stores the filtered result in df_filt

df_filt = df.groupby("family_accession").filter(lambda x: len(x) > 1200)

# Display the filtered DataFrame showing only families meeting the criteria
df_filt

## Create a balanced dataset

In [ ]:
# Create a balanced dataset by sampling 1200 sequences from each protein family
# 1. df_filt.groupby("family_accession") - Groups the filtered DataFrame by protein family accession
# 2. .apply(lambda x: x.sample(1200)) - For each family group:
#    - Takes a random sample of exactly 1200 sequences (using pandas sample())
#    - Ensures equal representation from each family
# 3. Stores the balanced dataset in df_bal

df_bal = df_filt.groupby("family_accession").apply(lambda x: x.sample(1200))

# Verify the balancing by counting sequences per family
# Shows the number of samples for each family (should all be 1200)
df_bal.family_accession.value_counts()

In [ ]:
# Peek into the balanced dataset
df_bal.head()

## Prepare input dataframe for modeling

The **reset_index**() function in pandas is used to restore a DataFrame's index to the default integer-based sequence (0 to n-1, where n represents the total number of rows) or to flatten a multi-level hierarchical index. When applied, this operation converts the existing index into a regular column within the DataFrame, unless explicitly instructed otherwise using the drop=True parameter. This function is particularly valuable for restructuring DataFrames after operations that modify the index, such as grouping, pivoting, or sorting, as it returns the data to a simpler, more conventional format. By transforming indices into columns, reset_index() enables easier data manipulation and analysis, especially when working with aggregated results or preparing data for visualization. The function also provides control over index preservation through its parameters, allowing users to either retain the original index as a new column or discard it entirely, making it a versatile tool for data cleaning and reorganization in pandas workflows.

In [ ]:
# Create a reduced DataFrame containing only the essential columns for analysis:
# 1. Selects only two columns from the balanced DataFrame (df_bal):
#    - "family_accession": The protein family identifier
#    - "sequence": The amino acid sequence data
# 2. Resets the index to default integer-based indexing:
#    - drop=True ensures the old index is discarded (not added as a column)
# 3. Stores the result in df_red (short for "reduced DataFrame")

df_red = df_bal[["family_accession", "sequence"]].reset_index(drop=True)

# Display the first 5 rows of the reduced DataFrame to verify the structure
df_red.head()

Calculate the number of distinct classes present in the dataset.

In [ ]:
# Calculate the number of unique protein family classes in the reduced dataset:
# 1. df_red.family_accession.value_counts() - Counts occurrences of each family accession
#    (returns a Series with family_accessions as index and counts as values)
# 2. len() - Counts the number of unique family accessions (length of the Series index)
# 3. Stores the result in num_classes (total number of distinct protein families)

num_classes = len(df_red.family_accession.value_counts())

# Display/return the number of classes (for verification or use in subsequent code)
num_classes

Calculate the total number of distinct Pfam family accession IDs present in the data.

In [ ]:
# Count the occurrences of each unique value in the 'family_accession' column of the dataframe 'df_red'
df_red.family_accession.value_counts()

### Make train and test datasets

Split the data into 75% for training (X_train) and 25% for testing (X_test). Further divide the test set equally: 50% for validation (X_val) and 50% for final testing (X_test).

In [ ]:
# Splits the DataFrame 'df_red' into training set 'X_train' and a temporary test set 'X_Test', allocating 25% of the data to the test set.
X_train, X_Test = train_test_split(df_red, test_size=0.25)

# Splits the temporary test set 'X_Test' further into validation set 'X_val' and final test set 'X_test', allocating 50% of 'X_Test' to the final test set.
X_val, X_test = train_test_split(X_Test, test_size=0.50)

Calculate the number of samples in the training, test, and validation splits to confirm dataset distribution aligns with expected ratios (e.g., 70-15-15).

In [ ]:
print(X_train.shape)
print(X_test.shape)
print(X_val.shape)

### Create amino acid sequence dictionary

In [ ]:
aa_seq_dict = {
    "A": 1,
    "C": 2,
    "D": 3,
    "E": 4,
    "F": 5,
    "G": 6,
    "H": 7,
    "I": 8,
    "K": 9,
    "L": 10,
    "M": 11,
    "N": 12,
    "P": 13,
    "Q": 14,
    "R": 15,
    "S": 16,
    "T": 17,
    "V": 18,
    "W": 19,
    "Y": 20,
}

In [ ]:
# Encode amino acid sequence using the dictionary above
# Define a function named 'aa_seq_encoder' that takes 'data' as input.
def aa_seq_encoder(data):
    # Initialize an empty list 'full_sequence_list' to store encoded sequences.
    full_sequence_list = []
    
    # Iterate over each sequence in the 'sequence' column of the input 'data' DataFrame.
    for i in data["sequence"].values:
        # Initialize an empty list 'row_sequence_list' for each individual sequence.
        row_sequence_list = []
        
        # Iterate over each amino acid 'j' in the current protein sequence 'i'.
        for j in i:
            # Look up the numerical encoding for amino acid 'j' in 'aa_seq_dict' and append it to 'row_sequence_list'. If not found, default to 0.
            row_sequence_list.append(aa_seq_dict.get(j, 0))
            
        # After processing all amino acids in a sequence, append the 'row_sequence_list' (now a NumPy array) to 'full_sequence_list'.
        full_sequence_list.append(np.array(row_sequence_list))
    # Return the 'full_sequence_list' containing encoded amino acid sequences as NumPy arrays.
    return full_sequence_list

# Encode the 'sequence' column in X_train DataFrame using the 'aa_seq_encoder' function and assign the result to 'X_train_encode'.
X_train_encode = aa_seq_encoder(X_train)

# Encode the 'sequence' column in X_val DataFrame using the 'aa_seq_encoder' function and assign the result to 'X_val_encode'.
X_val_encode = aa_seq_encoder(X_val)

# Encode the 'sequence' column in X_test DataFrame using the 'aa_seq_encoder' function and assign the result to 'X_test_encode'.
X_test_encode = aa_seq_encoder(X_test)

In [ ]:
# Show an example encoded amino acid sequence
X_train_encode[0]

The input sequences are padded to a fixed length of 100 tokens/values, ensuring uniform dimensions for downstream processing. 

In [ ]:
# Define the maximum length for padding/truncating protein sequences
# Sequences longer than this will be truncated, shorter ones will be padded
max_length = 100

# Pad/truncate the encoded training sequences:
# - X_train_encode: Pre-encoded training sequences (likely integer-encoded amino acids)
# - maxlen=max_length: Standardize all sequences to this length
# - padding="post": Add padding zeros AFTER the sequence content
# - truncating="post": Remove excess elements from the END if sequence exceeds max_length
X_train_padded = pad_sequences(
    X_train_encode, maxlen=max_length, padding="post", truncating="post"
)

# Apply identical padding/truncation to validation sequences
# Ensures consistent dimensions between training and validation data
X_val_padded = pad_sequences(
    X_val_encode, maxlen=max_length, padding="post", truncating="post"
)

# Apply identical padding/truncation to test sequences
# Maintains same preprocessing across all datasets
X_test_padded = pad_sequences(
    X_test_encode, maxlen=max_length, padding="post", truncating="post"
)

In [ ]:
# View original raw sequence string from training set (position 1)
X_train.sequence.iloc[1]

In [ ]:
# View encoded version (before padding) of the same sequence
# Shows integer mapping of amino acids
X_train_encode[1][:] 

In [ ]:
# View final padded/truncated version of the same sequence
# Shows how padding (zeros) were added to reach max_length
X_train_padded[1][:]

Convert target labels into numerical values ranging from `0` to `n_classes-1`, where n_classes represents the total number of unique categories. 

In [ ]:
# Initialize a LabelEncoder object to convert protein family labels (strings) to numerical indices
le = LabelEncoder()

# Encode training labels:
# 1. Fit the encoder on training family accessions (learns the label-to-index mapping)
# 2. Transform the labels to numerical indices (0 to n_classes-1)
y_train_enc = le.fit_transform(X_train["family_accession"])

# Encode validation labels using the SAME encoder (maintains consistent mapping)
y_val_enc = le.transform(X_val["family_accession"])

# Encode test labels using the SAME encoder (maintains consistent mapping)
y_test_enc = le.transform(X_test["family_accession"])

In [ ]:
# Show original family accession labels from training set
X_train["family_accession"]

In [ ]:
# Show corresponding encoded numerical labels
y_train_enc

In [ ]:
# Calculate total number of unique protein families (classes)
num_classes = len(le.classes_)
num_classes  # Display the count

In [ ]:
# Convert numerical labels to one-hot encoded vectors for neural network training:
# Each label becomes a binary vector of length num_classes with 1 at the class index
y_train = to_categorical(y_train_enc)  # Training set one-hot
y_val = to_categorical(y_val_enc)     # Validation set one-hot
y_test = to_categorical(y_test_enc)    # Test set one-hot

In [ ]:
# Display sample one-hot encoded training labels for verification
y_train

## Build model

In [ ]:
# Sequential groups a linear stack of layers into a tf.keras.Model.
# Sequential provides training and inference features on this model.
model = Sequential()

# EmbeddingLayer: Turns positive integers (indexes) into dense vectors of fixed size.
#  input_dim: Integer. Size of the vocabulary, i.e. maximum integer index + 1.
#  output_dim: Integer. Dimension of the dense embedding.
#  input_length: Length of input sequences, when it is constant.
model.add(Embedding(21, 16, name="EmbeddingLayer"))  # max_length not needed for Bidirectional LSTM

# Bidirectional wrapper for RNNs with 16 units of LSTM
model.add(Bidirectional(LSTM(16), name="BidirectionalLayer"))

# Applies Dropout to the input with 20% of the input units to drop.
model.add(Dropout(0.2, name="DropoutLayer"))

# densely-connected NN layer of 28 units
model.add(Dense(28, activation="softmax", name="DenseLayer"))

# Optimizer that implements the Adam algorithm
opt = tf.keras.optimizers.legacy.Adam(learning_rate=0.01)  # For TF 2.10+

# Configure the neural network model for training:
# - optimizer=opt: Uses the specified optimizer (e.g., Adam, SGD) for gradient descent
# - loss="categorical_crossentropy": Appropriate loss function for multi-class classification
# - metrics=["accuracy"]: Tracks accuracy during training/validation
model.compile(optimizer=opt, loss="categorical_crossentropy", metrics=["accuracy"])

# Display the model architecture summary:
# - Shows layer-by-layer structure
# - Displays output shapes and parameter counts
# - Provides total trainable parameters
model.summary()

**Early stopping** is a technique that halts model training when a monitored performance metric (like validation loss) stops improving, indicating that further training may lead to overfitting. This approach automatically finds the optimal stopping point by tracking the metric over epochs and terminating training when no improvement is observed for a specified number of consecutive iterations (patience). It serves as both an effective regularization method and computational efficiency measure.

In [ ]:
# monitor: Quantity to be monitored.
# Specifies the metric to monitor for early stopping (in this case, 'val_loss' - validation loss).
es = EarlyStopping(
    # patience: Number of epochs with no improvement after which training will be stopped.
    # Sets the number of epochs to wait after last time validation loss improved before stopping (patience=3).
    patience=3,
    
    # restore_best_weights: Whether to restore model weights from the epoch with the best value of the monitored quantity.
    # When set to True, restores model weights to the best epoch's weights when stopping (restore_best_weights=True).
    restore_best_weights=True
)

(The following cell may take a few minutes.)

In [ ]:
# Trains the model for a fixed number of epochs (iterations on a dataset)
# Training with fixed callbacks
# Calls the 'fit' method of the 'model' to train the neural network.
history = model.fit(
    # Provides the input training data (features) as 'X_train_padded'.
    X_train_padded,
    
    # Provides the target training data (labels) as 'y_train'.
    y_train,
    
    # Specifies the number of training epochs (full passes through the training data) as 10.
    epochs=10,
    
    # Sets the batch size for training, processing 256 samples at a time.
    batch_size=256,
    
    # Provides validation data (features and labels) as a tuple for monitoring performance on a separate dataset during training.
    validation_data=(X_val_padded, y_val),
    
    # Includes a list of callbacks, here using 'es' (likely an EarlyStopping callback) to control training process.
    callbacks=[es],  # Now using properly imported callback
    
    # Sets verbosity to 1 to display progress bars and training information during each epoch.
    verbose=1
)

### Plot accuracy and loss

In [ ]:
# Create a new figure for plotting with a size of 10x10 inches.
fig = plt.figure(figsize=(10, 10))

# Define the first subplot in a 2x2 grid (top-left subplot).
plt.subplot(2, 2, 1)

# Set the title of the subplot to "Accuracy" with a fontsize of 15.
plt.title("Accuracy", fontsize=15)

# Set the label for the x-axis to "Epochs" with a fontsize of 15.
plt.xlabel("Epochs", fontsize=15)

# Set the label for the y-axis to "Accuracy (%)" with a fontsize of 15.
plt.ylabel("Accuracy (%)", fontsize=15)

# Plot the validation accuracy from the training history, label it "Validation Accuracy", and use a dashed line style.
plt.plot(
    history.history["val_accuracy"], label="Validation Accuracy", linestyle="dashed"
)

# Plot the training accuracy from the training history and label it "Training Accuracy".
plt.plot(history.history["accuracy"], label="Training Accuracy")

# Display a legend in the lower right corner to distinguish between validation and training accuracy lines.
plt.legend(["Validation", "Training"], loc="lower right")

# Define the second subplot in a 2x2 grid (top-right subplot).
plt.subplot(2, 2, 2)

# Set the title of the subplot to "Loss" with a fontsize of 15.
plt.title("Loss", fontsize=15)

# Set the label for the x-axis to "Epochs" with a fontsize of 15.
plt.xlabel("Epochs", fontsize=15)

# Set the label for the y-axis to "Loss" with a fontsize of 15.
plt.ylabel("Loss", fontsize=15)

# Plot the validation loss from the training history, label it "Validation loss", and use a dashed line style.
plt.plot(history.history["val_loss"], label="Validation loss", linestyle="dashed")

# Plot the training loss from the training history and label it "Training loss".
plt.plot(history.history["loss"], label="Training loss")

# Display a legend in the upper right corner to distinguish between validation and training loss lines.
plt.legend(["Validation", "Training"], loc="upper right")

## Generate predictions

In [ ]:
# Generates output predictions for the input samples using the trained model.
y_pred = model.predict(X_test_padded)

# Build a text report showing the main classification metrics using sklearn's classification_report.
print(
    classification_report(
        # Converts one-hot encoded true labels (y_test) back to class indices using argmax.
        np.argmax(y_test, axis=1),
        
        # Converts probability predictions (y_pred) to class indices using argmax.
        np.argmax(y_pred, axis=1),
        
        # Uses class names from the label encoder (le) to label the classes in the report.
        target_names=le.classes_,
    )
)

- **Support** is the number of actual occurrences of the class in the specified dataset. 
- **Macro avg** takes the arithmetic mean (aka unweighted mean). 
- **Weighted avg** takes the mean of all per-class while considering each class’s support.

### Show predicted values

In [ ]:
y_pred

### Confusion matrix

A confusion matrix is a fundamental tool for evaluating the performance of a classification model by comparing its predictions against the true labels. The following code computes this matrix by first converting the one-hot encoded true labels (y_test) and predicted probability distributions (y_pred) into their respective class indices using np.argmax(axis=1). This step is necessary because the raw outputs of neural networks are typically probability vectors, while the confusion matrix requires discrete class labels. The confusion_matrix function then tabulates the predictions, with rows representing the actual classes and columns representing the predicted classes. The diagonal entries of the matrix indicate correct predictions (true positives), while the off-diagonal entries reveal misclassifications (false positives and negatives). This matrix provides a granular view of model performance, highlighting which classes are well-classified and where the model struggles, making it invaluable for diagnosing issues like class imbalance or biased predictions. Beyond mere accuracy, the confusion matrix enables the calculation of metrics like precision, recall, and F1-score, offering a comprehensive assessment of the model's strengths and weaknesses across all classes. Visualizing this matrix as a heatmap further aids interpretation, allowing quick identification of problematic class confusions and guiding targeted improvements to the model or dataset.

In [ ]:
# Calculates the confusion matrix using true labels (y_test) and predicted labels (y_pred).
#   - y_true: np.argmax(y_test, axis=1) -  True class labels.
#     - y_test:  Represents the true labels, likely in one-hot encoded format (e.g., from test data).
#     - np.argmax(y_test, axis=1): Converts one-hot encoded y_test to categorical labels by finding the index of the maximum value along axis 1 (rows).
#   - y_pred: np.argmax(y_pred, axis=1) - Predicted class labels.
#     - y_pred: Represents the predicted probabilities or one-hot encoded predictions from the model.
#     - np.argmax(y_pred, axis=1): Converts probability predictions or one-hot encoded predictions to categorical labels by finding the index of the maximum probability along axis 1 (rows).
cf_matrix = confusion_matrix(np.argmax(y_test, axis=1), np.argmax(y_pred, axis=1))

The code below generates a visual representation of a confusion matrix using a heatmap plot to facilitate model performance analysis. First, it establishes the dimensions of the plot by setting the figure size to 15 inches wide and 10 inches tall through plt.figure(figsize=(15, 10)), ensuring adequate space for clear visualization of the matrix elements. Using Seaborn's heatmap function, it then creates the heatmap visualization with several key parameters: the cf_matrix variable supplies the confusion matrix data to be plotted; annot=True enables the display of numerical values within each cell of the heatmap, allowing for precise interpretation of the counts in each category; fmt="" applies default formatting to these annotations; and cmap="Blues" selects a gradient of blue shades to represent the magnitude of values in the matrix, with darker blues typically indicating higher values. This visualization effectively highlights the model's classification patterns, making it easy to identify correct predictions along the diagonal and misclassifications in the off-diagonal cells, thereby providing an intuitive and informative summary of the model's performance across different classes. The combination of numerical annotations and color gradients allows for quick assessment of both the distribution and relative frequency of correct and incorrect classifications.

In [ ]:
# Set the figure size for the heatmap plot to 15 inches wide and 10 inches tall.
plt.figure(figsize=(15, 10))
# Create a heatmap using seaborn's heatmap function.
#   - cf_matrix: The confusion matrix data to be visualized as a heatmap.
#   - annot=True: Display numerical values (annotations) in each cell of the heatmap.
#   - fmt="":  Format string for the annotations (empty string means default formatting).
#   - cmap="Blues": Use the "Blues" colormap for the heatmap, representing values with shades of blue.
sns.heatmap(cf_matrix, annot=True, fmt="", cmap="Blues")

## Conclusion

Through this tutorial, we learned how to:
- Process and prepare protein sequence data for deep learning
- Implement a deep learning model for protein family classification
- Train and evaluate the model's performance
- Visualize and interpret the results using various metrics

The model achieved good classification performance across multiple protein families, demonstrating the effectiveness of deep learning approaches for protein sequence analysis.

## Clean up

Remember to shut down your Jupyter Notebook environment and delete any unnecessary files or resources once you've completed the tutorial.
